<center><img src = "https://juliacomputing.com/assets/img/new/JuliaDB_logo2.svg" width=400>

# A data system for Julia
</center>
<br>
<table>
    <tr>
        <td style="width: 200px"><strong style="font-size:20px;">Jeff Bezanson<br>Shashi Gowda<br>Josh Day</strong></td>
        <td><img src = "https://juliacomputing.com/assets/img/new/julia-computing.svg" width=400></td>
    </tr>
</table>


# Overview

- Why Julia
- Why JuliaDB
- Analytics with JuliaDB
- Benchmarks
- API Overview

## Why Do We Need Another Language?

https://julialang.org/blog/2012/02/why-we-created-julia

# The Two Language Problem

- **Prototype** code goes into a high-level language
- **Production** code goes into a low-level language

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Julia_prog_language.svg/1280px-Julia_prog_language.svg.png" width=200></center>

# The dream:

- Be (at least) as nice as Python/R/Matlab

- Be as fast as C/Fortran

*Julia is fast because of features which work well together, and it is more than "Fast Python/R/Matlab"*

# Julia Features

- **JIT Compiler**: User code runs fast
- **Multiple Dispatch**:  Code specialized to argument types
- **Type System**: Express yourself
- **Metaprogramming**: Transform and generate code
- **Parallelism**: Built-in
- **Interop** (Call C directly, RCall.jl, PyCall.jl), **Unicode Support**, **MIT Licensed**, ...

# Introducing JuliaDB

# Julia "Developers"

![](https://pkg.julialang.org/img/allver.svg)

# Julia "Users"

![](https://pkg.julialang.org/img/stars.svg)

# Why JuliaDB?

We shouldn't need to glue together tools to get performance

<img src="https://media.giphy.com/media/xT0xelg3s22Ni7gYO4/giphy.gif" width=400>

<table>
    <tr>
        <td width="100"><h1>pandas</h1></td>
        <td><img src = "images/pandas.png"></td>
    </tr>
</table>

<table>
    <tr>
        <td width="100"><h1>dplyr</h1></td>
        <td><img src = "images/dplyr.png"></td>
    </tr>
</table>

<table>
    <tr>
        <td width="100"><h1>data.table</h1></td>
        <td><img src = "images/datatable.png"></td>
    </tr>
</table>

<table>
    <tr>
        <td width="100"><h1>JuliaDB</h1></td>
        <td><img src = "images/juliadb.png"></td>
    </tr>
</table>

# JuliaDB Goals

**JuliaDB brings the promise of Julia to Data Science**

- Efficiently work with multi-file persistent datasets
- Queries/user-defined functions are fast
- Perform the heavy lifting of distributed computing
- Batteries are included (tools for analytics)

# Example: NYC Taxi Data

![](http://www.nyc.gov/html/tlc/includes/site_images/branding/banner.gif)

- First 4 months of yellow cab data for 2017 (3.2 GB)
- One file per month

In [32]:
;ls /Users/joshday/datasets/taxi

yellow_tripdata_2017-01.csv
yellow_tripdata_2017-02.csv
yellow_tripdata_2017-03.csv
yellow_tripdata_2017-04.csv


In [33]:
;du -h /Users/joshday/datasets/taxi

4.0K	/Users/joshday/datasets/taxi/.juliadb
3.2G	/Users/joshday/datasets/taxi


In [34]:
;head /Users/joshday/datasets/taxi/yellow_tripdata_2017-01.csv

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount

1,2017-01-09 11:13:28,2017-01-09 11:25:45,1,3.30,1,N,263,161,1,12.5,0,0.5,2,0,0.3,15.3
1,2017-01-09 11:32:27,2017-01-09 11:36:01,1,.90,1,N,186,234,1,5,0,0.5,1.45,0,0.3,7.25
1,2017-01-09 11:38:20,2017-01-09 11:42:05,1,1.10,1,N,164,161,1,5.5,0,0.5,1,0,0.3,7.3
1,2017-01-09 11:52:13,2017-01-09 11:57:36,1,1.10,1,N,236,75,1,6,0,0.5,1.7,0,0.3,8.5
2,2017-01-01 00:00:00,2017-01-01 00:00:00,1,.02,2,N,249,234,2,52,0,0.5,0,0,0.3,52.8
1,2017-01-01 00:00:02,2017-01-01 00:03:50,1,.50,1,N,48,48,2,4,0.5,0.5,0,0,0.3,5.3
2,2017-01-01 00:00:02,2017-01-01 00:39:22,4,7.75,1,N,186,36,1,22,0.5,0.5,4.66,0,0.3,27.96
1,2017-01-01 00:00:03,2017-01-01 00:06:58,1,.80,1,N,162,161,1,6,0.5,0.5,1.45,0,0.3,8.75


In [35]:
if nprocs() == 1
    @show addprocs(4)
end
using JuliaDB


path = "/Users/joshday/datasets/taxi"
db = loadtable(path; indexcols=[1, 2])
save(db, "mydata")

Metadata for 4 / 4 files can be loaded from cache.


Distributed Table with 39219765 rows in 4 chunks:
Columns:
#   colname                type
───────────────────────────────────
1   VendorID               Int64
2   tpep_pickup_datetime   DateTime
3   tpep_dropoff_datetime  DateTime
4   passenger_count        Int64
5   trip_distance          Float64
6   RatecodeID             Int64
7   store_and_fwd_flag     String
8   PULocationID           Int64
9   DOLocationID           Int64
10  payment_type           Int64
11  fare_amount            Float64
12  extra                  Float64
13  mta_tax                Float64
14  tip_amount             Float64
15  tolls_amount           Float64
16  improvement_surcharge  Float64
17  total_amount           Float64

# Run This Once:

```julia
addprocs()
using JuliaDB

path = "/Users/joshday/datasets/taxi"

db = loadtable(path; indexcols=[1, 2])

save(db, "mydata")
```

### Future Julia Sessions  Quickly Reload Data

In [36]:
@time db = load("mydata")

  0.008048 seconds (3.31 k allocations: 177.506 KiB)


Distributed Table with 39219765 rows in 4 chunks:
Columns:
#   colname                type
───────────────────────────────────
1   VendorID               Int64
2   tpep_pickup_datetime   DateTime
3   tpep_dropoff_datetime  DateTime
4   passenger_count        Int64
5   trip_distance          Float64
6   RatecodeID             Int64
7   store_and_fwd_flag     String
8   PULocationID           Int64
9   DOLocationID           Int64
10  payment_type           Int64
11  fare_amount            Float64
12  extra                  Float64
13  mta_tax                Float64
14  tip_amount             Float64
15  tolls_amount           Float64
16  improvement_surcharge  Float64
17  total_amount           Float64

# Analytics With OnlineStats
 
 http://joshday.github.io/OnlineStats.jl/latest/
 
- Each statistic/model has its own type
- Values are updated one observation at a time

# OnlineStats (Batteries included)

- **Summary Statistics** (`Mean`, `CovMatrix`, `Quantile`, etc.)
- **Density Estimation** (`OHistogram`, `IHistogram`, `FitCategorical`)
- **Regression** (`LinReg`, `LinRegBuilder`)
- **(Approximate) Penalized GLMs** with a variety of:
    - **Algorithms**: `SGD`, `ADAGRAD`, `ADAM`,...
    - **Losses**: `L2DistLoss`, `L1HingeLoss`,...
    - **Penalties**: `ElasticNetPenalty`, ...

# OnlineStats Features

- Different weighting schemes to handle parameter drift

<center>
![](https://user-images.githubusercontent.com/8075494/27964296-c249baec-6305-11e7-89d0-9875d3bdab3e.gif)
</center>

# OnlineStats Features

- Calculations can be done in parallel

<center><img src="https://user-images.githubusercontent.com/8075494/32748459-519986e8-c88a-11e7-89b3-80dedf7f261b.png" width=500></center>

In [37]:
using Plots, OnlineStats
plotly()  # Use plotly backend

Plots.PlotlyBackend()

# `reduce` Operations

In [38]:
reduce(+, db; select = :fare_amount) / length(db)

12.711173323960018

In [39]:
@time reduce(Mean(), db; select = :fare_amount)

  0.315611 seconds (12.48 k allocations: 651.479 KiB)


▦ Series{0}  |  EqualWeight  |  nobs = 39219765
└── Mean(12.7112)

In [40]:
@time s = reduce(FitCategorical(Int64), db; select = :passenger_count)
plot(s, xlab = "Number of Passengers")

  0.768730 seconds (15.59 k allocations: 833.419 KiB)


In [ ]:
@time s = reduce(2 * OHistogram(0:.5:60), db; 
    select = (:fare_amount, :tip_amount))
plot(s, label = [:Fare :Tip], xlab = :Amount, title = "Fare and Tips")

# Benchmarks


bit.ly/juliadb-pydata-benchmarks

In [ ]:
benchtype = vcat(fill("loadtable", 2), fill("groupby", 5), fill("join", 6))
bench = ["Read CSV (DateTime)", 
    "Read CSV (String)", 
    "mean(VendorID)", 
    "count(dayofweek, passenger_count)",
    "count(UDF, passenger_count)", 
    "count(dayofweek, passenger_count, floor)", 
    "count(passenger_count, dayofweek)", 
    "Inner Join (String)", "Inner Join (Number)", "Outer Join (String)", "Outer Join (Number)",
    "Left Join (String)","Left Join (Number)"]
jdb = [16.8086,18.0826,11.797,47.192, 51.797,190.741, 5.207,41.59,12.40,44.43,15.06,42.88,13.34]
pan = [32.8733,29.6489,16.487,83.629,280.981,115.429,51.726,16.48,19.28,24.92,37.32,16.87,19.57]

tb = table(benchtype, bench, jdb, pan; names = [:benchgroup, :benchmark, :JuliaDB, :Pandas])

In [ ]:
ratio = JuliaDB.select(tb, (:JuliaDB, :Pandas) => t -> t[1] / t[2])
plt = scatter(JuliaDB.select(tb, :benchgroup), ratio, group=JuliaDB.select(tb, :benchmark),
    title = "JuliaDB vs. Pandas Benchmarks", size=(1000, 400), ylab = "Time Relative to Pandas");

In [ ]:
plt

# `loadtable` benchmarks

- NY Taxi file: yellow_trips_2016-01.csv -- 10.9 million rows, 19 columns
<br>
<table>
    <tr>
        <td style="width: 200px"><strong style="font-size:20px;">Benchmark</strong></td> 
        <td style="width: 200px"><strong style="font-size:20px;">JuliaDB</strong></td>
        <td style="width: 200px"><strong style="font-size:20px;">Pandas</strong></td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">Read CSV (DateTime)</strong></td>
        <td style="font-size:20px;">16.8086 s</td>
        <td style="font-size:20px;">32.8733 s</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">Read CSV (String)</strong></td>
        <td style="font-size:20px;">18.0826 s</td>
        <td style="font-size:20px;">29.6489 s</td>
    </tr>
</table>

# `groupby` Benchmarks

1.6 million rows of taxi data

<br>
<table>
    <tr>
        <td style="width: 200px"><strong style="font-size:20px;">Benchmark</strong></td> 
        <td style="width: 200px"><strong style="font-size:20px;">JuliaDB</strong></td>
        <td style="width: 200px"><strong style="font-size:20px;">Pandas</strong></td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">mean(VendorID)</strong></td>
        <td style="font-size:20px;">11.797 ms</td>
        <td style="font-size:20px;">16.487 ms</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">count(dayofweek, passenger_count)</strong></td>
        <td style="font-size:20px;">47.192 ms</td>
        <td style="font-size:20px;">83.629 ms</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">count(UDF, passenger_count)</strong></td>
        <td style="font-size:20px;">51.797 ms</td>
        <td style="font-size:20px;">280.981 ms</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">count(dayofweek, passenger_count, floor)</strong></td>
        <td style="font-size:20px;">190.741 ms</td>
        <td style="font-size:20px;">115.429 ms</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">count(dayofweek, passenger_count, floor(Int))</strong></td>
        <td style="font-size:20px;">95.103 ms</td>
        <td style="font-size:20px;">NA</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">count(passenger_count, dayofweek)</strong></td>
        <td style="font-size:20px;">5.207 ms</td>
        <td style="font-size:20px;">51.726 ms</td>
    </tr>
</table>

# Join Benchmarks

**left:** 80k keys with 8k with uniques

**right:** 8k keys where 6k are present in left

**Key types:** 2 string fields, 2 floating point fields

<br>
<table>
    <tr>
        <td style="width: 200px"><strong style="font-size:20px;">Benchmark</strong></td> 
        <td style="width: 200px"><strong style="font-size:20px;">JuliaDB (String)</strong></td>
        <td style="width: 200px"><strong style="font-size:20px;">Pandas (String)</strong></td>
        <td style="width: 200px"><strong style="font-size:20px;">JuliaDB (Number)</strong></td>
        <td style="width: 200px"><strong style="font-size:20px;">Pandas (Number)</strong></td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">Inner Join</strong></td>
        <td style="font-size:20px;">41.59 ms</td>
        <td style="font-size:20px;">16.48 ms</td>
        <td style="font-size:20px;">12.40 ms</td>
        <td style="font-size:20px;">19.28 ms</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">Outer Join</strong></td>
        <td style="font-size:20px;">44.43 ms</td>
        <td style="font-size:20px;">24.92 ms</td>
        <td style="font-size:20px;">15.06 ms</td>
        <td style="font-size:20px;">37.32 ms</td>
    </tr>
    <tr>
        <td style="width: 300px"><strong style="font-size:20px;">Left Join</strong></td>
        <td style="font-size:20px;">42.88 ms</td>
        <td style="font-size:20px;">16.87 ms</td>
        <td style="font-size:20px;">13.34 ms</td>
        <td style="font-size:20px;">19.57 ms</td>
    </tr>
</table>

# API Overview

## Data structures

- **Table** -- a collection of rows, ordered by some fields
  - Good for analytics workloads
- **NDSparse** -- N-Dimensional sparse array
  - Good for scientific computing workloads

# API Overview

```
reindex, select, map, filter, dropna, columns, rows

setcol, pushcol, popcol, insertcol, insertcolafter, insertcolbefore, renamecol

reduce, groupreduce, groupby, flatten

join, groupjoin, asofjoin, merge

save, load
```

## Table specific

`loadtable`

## NDSparse specific

`loadndsparse, keys, values, selectkeys, selectvalues, reducedim, broadcast`

# Selection


In [ ]:
using Interact, JuliaDB
tbl = table([0.01,0.05,0.07], [1,2,3], [6,5,4], names=[:t, :x, :y])

selectors = split("""
    :x
    (:x, :y)
    :x => -
    (:x, :y) => p -> hypot(p.x, p.y)
    [1,2,3]
    (:x, :z => [7,8,9])
    (:t, :r => (:x, :y) => p -> hypot(p.x, p.y))""", "\n")


opts = Interact.selection(selectors)

result = map(signal(opts)) do sel
    code = "JuliaDB.select(tbl, $sel)"
    HTML("<pre>
        <strong>
        select(tbl, <span style='color:green'>$sel</span>)</strong>\n\n" * repr(
        eval(parse("JuliaDB.$code"))) * "</pre>"
    )
    end;

In [ ]:
display(tbl)
display(opts)
display(result)

# Selection

Wildly useful.

<code>select(t, <span style="color: green">which</span>)
map(f, t; <span style="color: green">select</span>)
reduce(f, t; <span style="color: green">select</span>)
filter(f, t; <span style="color: green">select</span>)
groupby(f, t, <span style="color: green">by</span>; <span style="color: green">select</span>)
groupreduce(f, t, <span style="color: green">by</span>; <span style="color: green">select</span>)
join(f, l, r; how, <span style="color: green">lkey</span>, <span style="color: green">rkey</span>, <span style="color: green">lselect</span>, <span style="color: green">rselect</span>)
groupjoin(f, l, r;how,  <span style="color: green">lkey</span>, <span style="color: green">rkey</span>, <span style="color: green">lselect</span>, <span style="color: green">rselect</span>)
</code>


(Green arguments are selectors)

# NDSparse

Key-value store that behaves like an n-dimensional array

In [ ]:
path = "/Users/joshday/datasets/pydatanyc/truefx"
X = loadndsparse(path,
        indexcols=[1,2],
        header_exists=false,
        distributed=false,
        colnames=["pair", "time", "bid", "ask"])

# Indexing by arbitrary keys

In [ ]:
keytype(X)

In [ ]:
eltype(X)

In [ ]:
X["AUD/JPY", DateTime("2016-12-30T00:00:00.032")]

In [ ]:
using IntervalSets

In [ ]:
t = table(X)
eltype(t)

In [ ]:
t[1]

# Indexing by arbitrary keys

In [ ]:
X[:, Date("2016-12-30")..Date("2016-12-31")]

# Indexing by arbitrary keys

In [ ]:
X[["AUD/USD", "CAD/JPY"], Date("2016-12-30")..Date("2016-12-31")]

# Indexing by arbitrary keys

In [ ]:
bids = selectvalues(X, :bid)

# Behaves like N-Dimensional Array

In [ ]:
eltype(bids)

In [ ]:
reducedim(+, bids, 2)

# Behaves like N-Dimensional Array

In [ ]:
# Aggregate maximum bid for every hour

hourlybids = convertdim(bids, :time, x->trunc(x, Dates.Hour),
                        agg=max)

# Behaves like N-Dimensional Array

In [ ]:
audjpy = hourlybids["AUD/JPY", :]
audjpy = selectkeys(audjpy, (:time,)) # squeeze

# Behaves like N-Dimensional Array

In [ ]:
hourlybids ./ audjpy

# Behaves like N-Dimensional Array

In [ ]:
broadcast(hypot, hourlybids, audjpy)

# NDSparse -- summary

- indexing by arbitrary keys or intervals of values
- iterates by values
- behaves like multi-dimensional arrays
- supports array syntax like broadcast, reducedim, map, reduce
- great for any "Series" -- esp. time series

## Onward!

- Feature extraction for machine-learning
- More comprehensive out-of-core support
- Dense ND data store (e.g. satellite imagery: X, Y, Z, T -> R, G, B)
- Streaming updates

# Thank You

<center><img src = "https://juliacomputing.com/assets/img/new/JuliaDB_logo2.svg" width=400>

</center>
<br>
<table>
    <tr>
        <td style="width: 200px"><strong style="font-size:20px;">jeff@juliacomputing.com<br>shashi@juliacomputing.com<br>josh@juliacomputing.com</strong></td>
        <td><img src = "https://juliacomputing.com/assets/img/new/julia-computing.svg" width=400></td>
    </tr>
</table>

- https://github.com/JuliaComputing/JuliaDB.jl
- https://juliacomputing.com